<center><font size='3'>Extraction :ALBERT😛 [Train + Infer]</font></center>
![](https://miro.medium.com/max/450/1*p3Ste5R_iJzi5IcSmFkmtg.png)
* This competition wants us to Extract the words which are responsible for the sentiment of the tweet.
* IMP POINT: Almost 97 % Jaccard Similarity in train data "text" and "selected_text".In conclusion maybe  we can use neutral "text" as it is for "selected_text" for test data submission. <br>


<font size='3' color='GREEN'>Contents</font>
* [Install and imports](#1)
* [Convert to JSON](#2)
* [MODEL TRAIN](#3)
* [MODEL INFER](#4)

This kernel is build from refing :https://www.kaggle.com/vaishvik25/question-answering-starter-pack

<center><font size='6' color='blue'> ALBERT (WHAT? AND WHY?) </font></center><br>


<font size='3'>
WHAT ?<br>The ALBERT model was proposed in ALBERT: A Lite BERT for Self-supervised Learning of Language Representations by Zhenzhong Lan, Mingda Chen, Sebastian Goodman, Kevin Gimpel, Piyush Sharma, Radu Soricut. It presents two parameter-reduction techniques to lower memory consumption and increase the trainig speed of BERT:<br>
] Splitting the embedding matrix into two smaller matrices<br>
] Using repeating layers split among groups<br>
] https://arxiv.org/abs/1909.11942 <br>
</font>
<br><br>
<font size='3'>
WHY ?<br> Its Currently Trending on SQuAD2.0 Leaderboard for Question Answering Model F1 Accuracy. So, may be for Q&A apporch it can give nice result.
</font>
![](https://i.ibb.co/tsJm3fj/albert.png) 

# <a name="imports" id="1"></a>1. Install and Import

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
import pandas as pd
import numpy as np
import json

In [ ]:
train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
sub_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

train = np.array(train_df)
test = np.array(test_df)
use_cuda = True

# <a name="reading" id="2"></a> 2. Converting test data to JSON

* Train data is already converted here. https://www.kaggle.com/vaishvik25/tweet-sentiment-extraction-json<br>
* But during submission kaggle use full test set there it is needed to convert in kernel. <br>
* The Code of JSON convertion from https://www.kaggle.com/jonathanbesomi/question-answering-starter-pack

In [ ]:
%%time

"""
Prepare testing data in QA-compatible format
"""

output = {}
output['version'] = 'v1.0'
output['data'] = []

def do_qa_test(test):
    paragraphs = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        paragraphs.append({'context': context.lower(), 'qas': qas})
        output['data'].append({'title': 'None', 'paragraphs': paragraphs})
    return paragraphs

qa_test = do_qa_test(test)

with open('test.json', 'w') as outfile:
    json.dump(qa_test, outfile)


# <a name="Training" id="3"></a> 3. Model Training ....

Using my training kernel out to save GPU time.<br>
To train uncomment train command.

In [ ]:
MODEL_PATH = '/kaggle/input/albert-train/outputs/checkpoint-10308-epoch-3/' # MODEL PATH OF PRETRAINED MODEL


model = QuestionAnsweringModel('albert', 
                               MODEL_PATH, 
                                   args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    },
                              use_cuda=use_cuda)


# model.train_model('/kaggle/input/tweet-sentiment-extraction-json/train.json') #UNCOMMENT to TRAIN

# <a name="Infer" id="4"></a> 4. Model INFER

In [ ]:
%%time
# Infering on trained model
predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)

sub_df['selected_text'] = predictions_df['answer']
sub_df.to_csv('submission.csv', index=False)

sub_df.head()


<font size='3' color='Blue'>If you find this kernel useful, consider doing an upvote [^]</font>
![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSMkBhNBPXLA05LbULNGM--2DOdI4z_mQOcrBHkKoLaPA4BwgpI&usqp=CAU)